It's first version file for hitmo

In [1]:
import os
import requests
from bs4 import BeautifulSoup
import csv
import time

In [5]:
def get_track_ids_from_main():
    url = "https://rus.hitmotop.com"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        print("Main page для получения ID треков...")
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        track_ids = []
        track_elements = soup.select('a[href^="/song/"]')
        
        print(f"Найдено ссылок на треки: {len(track_elements)}")
        
        for link in track_elements:
            href = link.get('href')
            if href and href.startswith('/song/'):
                track_id = href.split('/song/')[-1]
                if track_id and track_id not in track_ids:
                    track_ids.append(track_id)
        
        print(f"Уникальных ID треков: {len(track_ids)}")
        return track_ids #можно поменять количество
        
    except Exception as e:
        print(f"Ошибка при получении ID: {e}")
        return []

In [6]:
def get_fallback_track_data(track_ids):
    print("\n Используем базовый парсинг страниц...")
    
    tracks_data = []
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        main_response = requests.get("https://rus.hitmotop.com", headers=headers)
        main_response.raise_for_status()
        main_soup = BeautifulSoup(main_response.content, 'html.parser')
        
        playlist_element = main_soup.find('h2', class_='tracks__title') or main_soup.find('h2', class_='tracks_title')
        playlist_name = playlist_element.get_text(strip=True) if playlist_element else 'Популярные треки'
        print(f"Название плейлиста: {playlist_name}")
        
    except Exception as e:
        print(f"Не удалось получить название плейлиста: {e}")
        playlist_name = 'Популярные треки'
    
    for i, track_id in enumerate(track_ids, 1):
        try:
            track_url = f"https://rus.hitmotop.com/song/{track_id}"
            print(f"Обрабатываем трек {i}/{len(track_ids)}: {track_id}")
            
            response = requests.get(track_url, headers=headers)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            title_element = soup.find('h1', class_='p-track-title')
            
            duration_element = soup.select_one('.track__fulltime')
            duration = duration_element.get_text(strip=True) if duration_element else 'N/A'
            
            if title_element:
                track_text = title_element.get_text(strip=True)
                
                if ' - ' in track_text:
                    artist, title = track_text.split(' - ', 1)
                else:
                    artist = "Неизвестно"
                    title = track_text
                
                tracks_data.append({
                    'number': i,
                    'artist': artist,
                    'title': title,
                    'track_id': track_id,
                    'duration': duration,
                    'playlist': playlist_name,
                    'rating': 'N/A',
                    'plays': 'N/A',
                    'url': track_url,
                    'image': ''
                })
                
                print(f"{i}. {artist} - {title} | Длительность: {duration} | Плейлист: {playlist_name}")
            else:
                print(f"{i}. Не удалось распарсить трек {track_id}")
            
            time.sleep(1)
            
        except Exception as e:
            print(f"Ошибка при парсинге трека {track_id}: {e}")
            continue
    
    return tracks_data

In [7]:
def save_to_csv(tracks_data, filename):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow([
                '№', 'Исполнитель', 'Название трека', 'ID трека', 
                'Длительность', 'Плейлист', 'Рейтинг', 'Прослушивания', 'Ссылка'
            ])
            
            for track in tracks_data:
                writer.writerow([
                    track['number'],
                    track['artist'],
                    track['title'],
                    track['track_id'],
                    track['duration'],
                    track['playlist'],
                    track['rating'],
                    track['plays'],
                    track['url']
                ])
        
        print(f"Данные сохранены в файл: {filename}")
        print(f"Полный путь: {os.path.abspath(filename)}")
        return True
        
    except Exception as e:
        print(f"Ошибка при сохранении в CSV: {e}")
        return False

In [8]:
def scrape_hitmotop_simple():
    print("=== HITMOTOP PARSER ===")
    print("Начинаем парсинг...")
    
    track_ids = get_track_ids_from_main()
    
    if not track_ids:
        print("Не удалось получить ID треков")
        return False
    
    tracks_data = get_fallback_track_data(track_ids)
    
    if not tracks_data:
        print("Не удалось получить данные треков")
        return False
    
    filename = 'hitmotop_tracks.csv'
    success = save_to_csv(tracks_data, filename)
    
    if success:
        print(f"\n УСПЕХ! Создан файл: {filename}")
        print(f"Обработано треков: {len(tracks_data)}")
        return True
    else:
        print("Не удалось сохранить данные")
        return False

def wait_for_exit():
    """Ждем нажатия Enter перед закрытием"""
    print("\n" + "="*50)
    print("Нажмите Enter для выхода...")
    input()

In [9]:
if __name__ == "__main__":
    try:
        success = scrape_hitmotop_simple()
        
        if success:
            print("\n Программа завершена успешно!")
        else:
            print("\n Программа завершена с ошибками")
        
    except Exception as e:
        print(f"\nКритическая ошибка: {e}")
    
    wait_for_exit()

=== HITMOTOP PARSER ===
Начинаем парсинг...
Main page для получения ID треков...
Найдено ссылок на треки: 37
Уникальных ID треков: 31

 Используем базовый парсинг страниц...
Название плейлиста: Популярные треки
Обрабатываем трек 1/31: 79186666
1. Артур Пирожков - Само Собой | Длительность: 02:38 | Плейлист: Популярные треки
Обрабатываем трек 2/31: 74376135
2. ANNA ASTI - По барам | Длительность: 03:58 | Плейлист: Популярные треки
Обрабатываем трек 3/31: 62788609
3. GAYAZOV$ BROTHER$ - Пьяный туман | Длительность: 03:54 | Плейлист: Популярные треки
Обрабатываем трек 4/31: 75704918
4. Кравц, Гио Пика - Где прошла ты | Длительность: 02:48 | Плейлист: Популярные треки
Обрабатываем трек 5/31: 62571704
5. Король и Шут - Лесник | Длительность: 03:12 | Плейлист: Популярные треки
Обрабатываем трек 6/31: 77552797
6. AP$ENT - Можно я с тобой | Длительность: 02:12 | Плейлист: Популярные треки
Обрабатываем трек 7/31: 78610932
7. Планета 9 - Фрау Марта | Длительность: 02:07 | Плейлист: Популярные тр